In [1]:
import os
import sys

import warnings
warnings.filterwarnings('ignore')

curruser = os.environ.get('USER')
sys.path.insert(0, './src/')
# sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser)) 
# sys.path.insert(0, '/home/{}/python36-libs/lib/python3.6/site-packages/'.format(curruser))

In [2]:
from pathlib import Path
import re
import pandas as pd
import numpy as np
import random
from tqdm import tqdm, tqdm_notebook
from sklearn.preprocessing import LabelEncoder

In [3]:
import numpy as np
np.random.seed(42)

import tensorflow as tf
tf.set_random_seed(42)
# import keras as K
from keras import backend as K
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Activation, Permute
from keras.layers import LSTM, Dropout, CuDNNGRU, Bidirectional
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop, Adam, Adagrad, Adadelta, SGD
from keras.layers import Input, Flatten, Dense, Dropout, Convolution1D, Conv1D, \
                         MaxPool1D, Lambda, GlobalMaxPooling1D, GlobalAveragePooling1D, \
                         BatchNormalization, Activation, AveragePooling1D, Concatenate, concatenate

from sklearn.model_selection import train_test_split, cross_val_score, \
                                    KFold, StratifiedKFold, RepeatedKFold, RepeatedStratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from keras.utils import np_utils, to_categorical
import keras
import math        
        
import matplotlib.pyplot as plt
import pickle
import sys
import heapq
import seaborn as sns
from pylab import rcParams

Using TensorFlow backend.


In [4]:
%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 12, 5


def plot_metrics(history):
    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    
    plt.figure()
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')

In [5]:
with K.tf.device('/GPU:0'):
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.8
    session = tf.Session(config=config)
    K.set_session(session)
#     set_gpu_option("0", 0.4)

In [29]:
from tensorflow.python.client import device_lib
from keras.backend.tensorflow_backend import set_session

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

def set_gpu_option(which_gpu, fraction_memory):
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = fraction_memory
    config.gpu_options.visible_device_list = which_gpu
    set_session(tf.Session(config=config))
    return

In [ ]:
# Preventing TF from allocating the totally of a GPU memory
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [ ]:
get_available_gpus()

In [ ]:
set_gpu_option("0", 0.4)

# Preprocessing

### Fetch data from csv

In [30]:
import csv
csvpath = Path.joinpath(Path(os.getcwd()),'data/dnaseg/csv/','DnaSeg4TrainwLabelsSampled_wRandL.csv')

In [31]:
df = pd.read_csv(csvpath, delimiter=';', encoding='utf8')

In [9]:
df.head()

,seq,label
0,ATATGTATTTTCTTTTTGTGGAGAGCATTTTTCCCTCGTGATTACA,0
1,TGGAGGGGAAAAGCCAGGAGACCTCCGAGCTTGCACATATTGTAGA,0
2,TTAACCTTGCAAGAACTCTTCAGGCACATATGGAAGATCTCG,0
3,TCTGAAGTAAATTATATCATTGAAAGACCAAGCTACCCTCTGAAGA...,0
4,GACTTGCATACCAACATAATCAGACCGTCTGCAGAAATTCTCCTAC...,0


In [12]:
df.shape

(778376, 2)

### Don't Forget to Shuffle Train Data

In [32]:
df = df.sample(frac=1, random_state=42)

In [33]:
seqlst = df.seq.tolist()

In [34]:
chars = sorted(list(set(seqlst[0])))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [35]:
char_indices

{'A': 0, 'C': 1, 'G': 2, 'T': 3}

### Create one-hot encoded vectors using the `char_indices` map

In [36]:
SEQUENCE_LENGTH = df.seq.str.len().max()
step = 5

sentences = df.seq.tolist()
next_chars = df.label.tolist()

In [37]:
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)),dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in tqdm(enumerate(sentences)):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
        #y[i, char_indices[next_chars[i]]] = 1

778376it [00:14, 54368.45it/s]


In [38]:
y = to_categorical(next_chars, num_classes=4)

In [15]:
X.shape

(778376, 119, 4)

In [39]:
del sentences, next_chars, df

### UDC - User Defined Callbacks

### Learning Rate Schedules and Adaptive Learning Rate

In [40]:
def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 4.0
    lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
    return lrate
lrate = LearningRateScheduler(step_decay)

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
       self.losses = []
       self.lr = []
 
    def on_epoch_end(self, batch, logs={}):
       self.losses.append(logs.get('loss'))
       self.lr.append(step_decay(len(self.losses)))

In [41]:
loss_history = LossHistory()
lrate = LearningRateScheduler(step_decay)
callbacks_list = [loss_history, lrate]

In [42]:
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.01, patience=patience_lr, 
                                       verbose=1, epsilon=1e-4, mode='min')
    return [mcp_save, reduce_lr_loss]

## Building Multi-Layered LSTM with TimeDistributed SubLayer

In [ ]:
model = Sequential()
# model.add(LSTM(units=128, input_shape=(2*SEQUENCE_LENGTH, len(chars)),
#                dropout=0.2, recurrent_dropout=0.2))
# model.add(Permute((2,1), input_shape=(2*SEQUENCE_LENGTH, len(chars))))
model.add(LSTM(units=128, input_shape=(2*SEQUENCE_LENGTH, len(chars)), 
               return_sequences=True,kernel_initializer = "he_uniform", use_bias = True,
               dropout=0.4, recurrent_dropout=0.4, activation="tanh"))
# model.add(TimeDistributed(Dense(10, input_shape = (2*SEQUENCE_LENGTH, 128), activation="tanh")))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2, activation="tanh", use_bias = True))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
indx = np.random.choice(np.arange(len(X)), size=1000000)
name_weights = "./models/keras/final_model" + "_weights.h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=4)
    
# optimizer = RMSprop(lr=0.06)
optimizer = Adam(lr=0.01)
# optimizer = Adagrad(lr=0.02, decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X[indx], y[indx], 
                    validation_split=0.05, 
                    batch_size=1024, 
                    epochs=10,
                    callbacks=callbacks,
                    shuffle=True).history

In [ ]:
plot_metrics(history)

## Try to Outperform Targeted Metrics Using Cross-Val Technique

In [ ]:
name_weights = "./models/keras/final_model_fold" + "_weights.h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=4)

# optimizer = RMSprop(lr=0.06)
# optimizer = Adam(lr=0.01)
optimizer = Adagrad(lr=0.05, decay=0.01)

def buildmodel():
    model = Sequential()
    # model.add(LSTM(units=128, input_shape=(2*SEQUENCE_LENGTH, len(chars)),
    #                dropout=0.2, recurrent_dropout=0.2))
    model.add(LSTM(units=128, input_shape=(2*SEQUENCE_LENGTH, len(chars)), return_sequences=True,
                   dropout=0.3, recurrent_dropout=0.3))
    model.add(TimeDistributed(Dense(10, input_shape = (2*SEQUENCE_LENGTH,128), activation="relu")))
    model.add(LSTM(units=64, dropout=0.4, recurrent_dropout=0.4))
    model.add(Dense(units=32, activation='relu'))
    model.add(Dense(len(chars)))
    model.add(Activation('softmax'))    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return(model)

In [ ]:
buildmodel().summary()

### ScikitWrapper4Keras: Implementation of the scikit-learn classifier API for Keras.

In [ ]:
estimator = KerasClassifier(build_fn=buildmodel, epochs=15, batch_size=1024, verbose=1)
kfold = KFold(n_splits=4, shuffle=True)
results = cross_val_score(estimator, X, y, cv=kfold, n_jobs=1) 

### Use Different Folding Techniques To Achieve Best Metrics (by preserving the percentage of samples for each class)

In [ ]:
folds = RepeatedStratifiedKFold(n_splits=4, n_repeats=20, random_state=45).split(X, y.argmax(1))

In [ ]:
batch_size=1024

for j, (train_idx, val_idx) in enumerate(folds):
    
    print('Fold: {}'.format(j))
    X_train_cv = X[train_idx]
    y_train_cv = y[train_idx]
    X_valid_cv = X[val_idx]
    y_valid_cv = y[val_idx]
    
    buildmodel().fit(
                X_train_cv,
                y_train_cv,
                batch_size = batch_size,
                #steps_per_epoch = int(len(X_train_cv)/batch_size),
                #validation_steps = int(len(X_valid_cv)/batch_size),
                epochs=15,
                shuffle=True,
                verbose=1,
                validation_data = (X_valid_cv, y_valid_cv),
                callbacks = callbacks)
    
    print(buildmodel().evaluate(X_valid_cv, y_valid_cv))      

## Straight-Forward STACKING: Try to perform a Stacking of different LSTM layers

In [ ]:
model = Sequential()
model.add(LSTM(units=128, input_shape=(2*SEQUENCE_LENGTH, len(chars)), return_sequences=True,
                                       dropout=0.4, recurrent_dropout=0.4))
model.add(LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
# optimizer = RMSprop(lr=0.01)
optimizer = Adagrad(lr=0.1, decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, y, validation_split=0.05, batch_size=128, epochs=10, shuffle=True).history

In [ ]:
plot_metrics(history)

## LSTM+FCN for Time Series Classification (Fully Convolutional Block)

![alt text](./img/20181125-lstm-fcn_architecture.png "LSTM FCN for Time Series Classification")

In [43]:
NB_CLASSES = 4
SEQUENCE_LENGTH = X.shape[1]
SEQUENCE_LENGTH

119

In [44]:
def lstm_fcn_block():
    ip    = Input(shape = (SEQUENCE_LENGTH, NB_CLASSES))
#     iphat = Permute((2,1), input_shape=(2*SEQUENCE_LENGTH, NB_CLASSES))(ip)
    x = LSTM(units=128, input_shape=(SEQUENCE_LENGTH, NB_CLASSES), 
             return_sequences=True, kernel_initializer = "he_uniform",
             dropout=0.4, recurrent_dropout=0.4, activation="tanh")(ip)
#     for i in range(3):
#         if i==0:
#             x = LSTM(units=128, input_shape=(SEQUENCE_LENGTH, NB_CLASSES), 
#                      return_sequences=True, kernel_initializer = "he_uniform",
#                      dropout=0.4, recurrent_dropout=0.4, activation="tanh")(ip)   
# #             x = TimeDistributed(Dense(10, input_shape = (SEQUENCE_LENGTH,128), activation="tanh"))(x)            
#         else:
#             x = Concatenate()([x, ip])
#             x = LSTM(units=128, 
#                      return_sequences=True, kernel_initializer = "he_uniform",
#                      dropout=0.4, recurrent_dropout=0.4, activation="tanh")(x)

    x = LSTM(units=64, dropout=0.4, recurrent_dropout=0.4, 
             kernel_initializer = "he_uniform", activation="tanh")(x)
#     x = Dense(units=32, activation='relu')

    rnn, rnn_fw, rnn_bw = Bidirectional(CuDNNGRU(100, return_sequences=False, return_state=True))(ip)
    
#     iphat = Permute((2,1), input_shape=(SEQUENCE_LENGTH, NB_CLASSES))(ip)
    y = Conv1D(filters=256, kernel_size=3, strides=1, 
               padding='valid', use_bias=True, 
#                data_format='channels_first', 
               kernel_initializer = "he_uniform", 
               name='c1d_1')(ip)
    y = Activation("tanh")(y)
    y = MaxPool1D(pool_size=2)(y)    
#     y = MaxPool1D(pool_size=2, data_format='channels_first')(y)
#     y = BatchNormalization()(y)
    
    y = Conv1D(filters=256, kernel_size=3, strides=1, 
               padding='valid', use_bias=True, 
#                data_format='channels_first', 
               kernel_initializer = "he_uniform", 
               name='c1d_2')(y)
    y = Activation("relu")(y)
#     y = MaxPool1D(pool_size=3, data_format='channels_first')(y)
#     y = MaxPool1D(pool_size=2)(y)
#     y = BatchNormalization()(y)
    
    y = Conv1D(filters=256, kernel_size=3, strides=1, 
               padding='valid', use_bias=True, 
#                data_format='channels_first', 
               kernel_initializer = "he_uniform", 
               name='c1d_3')(y)
    y = Activation("tanh")(y)
#     y = MaxPool1D(pool_size=2, data_format='channels_first')(y)
    y = MaxPool1D(pool_size=2)(y)
#     y = BatchNormalization()(y)    
    
    y = Conv1D(filters=128, kernel_size=3, strides=1, 
               padding='valid', use_bias=True, 
#                data_format='channels_first', 
               kernel_initializer = "he_uniform", 
               name='c1d_4')(y)
    y = Activation("relu")(y)
#     y = BatchNormalization()(y)
    
    y = Flatten()(y)
#     y = GlobalAveragePooling1D()(y)
    y = Dropout(0.5)(y)
    x = Concatenate()([x, y, rnn])

    x = Dense(1024, activation = "relu")(x)
    x = Dropout(0.4)(x)
    x = Dense(1024, activation = "relu")(x)
    x = Dropout(0.3)(x)    
    
    out = Dense(NB_CLASSES, activation = "softmax")(x)

    model = Model(ip, out)

    model.summary()

    return model

In [45]:
model = lstm_fcn_block()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 119, 4)       0                                            
__________________________________________________________________________________________________
c1d_1 (Conv1D)                  (None, 117, 256)     3328        input_2[0][0]                    
__________________________________________________________________________________________________
activation_5 (Activation)       (None, 117, 256)     0           c1d_1[0][0]                      
__________________________________________________________________________________________________
max_pooling1d_3 (MaxPooling1D)  (None, 58, 256)      0           activation_5[0][0]               
__________________________________________________________________________________________________
c1d_2 (Con

In [46]:
# indx = np.random.choice(np.arange(len(X)), size=6000000)
name_weights = "./models/keras/cnn_bilstm_model.h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=4)
    
optimizer = SGD(lr=5e-2, momentum=0.5, decay=1e-2, nesterov=True)    
# optimizer = RMSprop(lr=0.04)
# optimizer = Adam(lr=5e-2, decay=5e-3)
# optimizer = Adadelta(lr=5e-2, decay=5e-3)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X[:], y[:], 
                    validation_split=0.1, 
                    batch_size=256, 
                    epochs=140,
                    callbacks=callbacks,
                    shuffle=True).history

Train on 700538 samples, validate on 77838 samples
Epoch 1/140
700538/700538 [==============================] - 1362s 2ms/step - loss: 1.3801 - acc: 0.2854 - val_loss: 1.3758 - val_acc: 0.2982
Epoch 2/140
700538/700538 [==============================] - 1361s 2ms/step - loss: 1.3774 - acc: 0.2919 - val_loss: 1.3752 - val_acc: 0.2997
Epoch 3/140
700538/700538 [==============================] - 1363s 2ms/step - loss: 1.3769 - acc: 0.2935 - val_loss: 1.3749 - val_acc: 0.2997
Epoch 4/140
700538/700538 [==============================] - 1361s 2ms/step - loss: 1.3765 - acc: 0.2944 - val_loss: 1.3745 - val_acc: 0.3009
Epoch 5/140
700538/700538 [==============================] - 1362s 2ms/step - loss: 1.3762 - acc: 0.2953 - val_loss: 1.3743 - val_acc: 0.3012
Epoch 6/140
700538/700538 [==============================] - 1364s 2ms/step - loss: 1.3760 - acc: 0.2952 - val_loss: 1.3742 - val_acc: 0.3020
Epoch 7/140
700538/700538 [==============================] - 1368s 2ms/step - loss: 1.3759 - acc:

KeyboardInterrupt: 

In [ ]:
model.save('./models/keras/bilstm_fcnn.h5')

## CNN with FCL Only

In [ ]:
def lstm_fcn_block():
    ip    = Input(shape = (2*SEQUENCE_LENGTH, NB_CLASSES))
    iphat = Permute((2,1), input_shape=(2*SEQUENCE_LENGTH, NB_CLASSES))(ip)
    y = Conv1D(filters=256, kernel_size=3, strides=1, 
               padding='valid', use_bias=True, 
               data_format='channels_first', 
               kernel_initializer = "he_uniform", 
               name='c1d_1')(iphat)
    y = Activation("relu")(y)
    y = MaxPool1D(pool_size=2, data_format='channels_first')(y)
#     y = BatchNormalization()(y)
    
    y = Conv1D(filters=256, kernel_size=3, strides=1, 
               padding='valid', use_bias=True, 
               data_format='channels_first', 
               kernel_initializer = "he_uniform", 
               name='c1d_2')(y)
    y = Activation("relu")(y)
    y = MaxPool1D(pool_size=2, data_format='channels_first')(y)
#     y = BatchNormalization()(y)
    
    y = Conv1D(filters=256, kernel_size=5, strides=1, 
               padding='valid', use_bias=True, 
               data_format='channels_first', 
               kernel_initializer = "he_uniform", 
               name='c1d_3')(y)
    y = Activation("relu")(y)
#     y = MaxPool1D(pool_size=2, data_format='channels_first')(y)
#     y = BatchNormalization()(y)    
 
    y = Conv1D(filters=256, kernel_size=2, strides=1, 
               padding='valid', use_bias=True, 
               data_format='channels_first', 
               kernel_initializer = "he_uniform", 
               name='c1d_4')(y)
    y = Activation("relu")(y)    
    
    y = Conv1D(filters=256, kernel_size=2, strides=1, 
               padding='valid', use_bias=True, 
               data_format='channels_first', 
               kernel_initializer = "he_uniform", 
               name='c1d_5')(y)
    y = Activation("relu")(y)
    
    y = Conv1D(filters=256, kernel_size=3, strides=1, 
               padding='valid', use_bias=True, 
               data_format='channels_first', 
               kernel_initializer = "he_uniform", 
               name='c1d_6')(y)
    y = Activation("relu")(y)
    y = MaxPool1D(pool_size=2, data_format='channels_first')(y)    
#     y = BatchNormalization()(y)
    
    y = Flatten()(y)
#     y = GlobalAveragePooling1D()(y)

    y = Dense(1024, activation = "relu")(y)
    y = Dropout(0.5)(y)
    y = Dense(1024, activation = "relu")(y)
    y = Dropout(0.5)(y)
    out = Dense(NB_CLASSES, activation = "softmax")(y)

    model = Model(ip, out)

    model.summary()

    return model

In [ ]:
model = lstm_fcn_block()

In [ ]:
# indx = np.random.choice(np.arange(len(X)), size=6000000)
name_weights = "./models/keras/final_model" + "_weights.h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=4)
    
optimizer = SGD(lr=0.1, nesterov=True)
# optimizer = Adam(lr=0.03)
# optimizer = Adadelta(lr=0.1, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])
history = model.fit(X[:], y[:], 
                    validation_split=0.05, 
                    batch_size=512, 
                    epochs=40,
                    callbacks=callbacks,
                    shuffle=True).history

# OOT: predict some character completions using our model!

In [ ]:
import csv
from itertools import compress
from LSTMPostProc import LSTM_pred

In [ ]:
model = load_model(filepath='./models/keras/bilstm_fcnn.h5')

In [ ]:
SEQUENCE_LENGTH = 30

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
csvpath = Path.joinpath(Path(os.getcwd()),'data/dnaseg/csv','DnaSeg4ValCharLabel.csv')

In [ ]:
df = pd.read_csv(csvpath, delimiter=';', encoding='utf8')
oobsents = df.seq.tolist()
oobchars = df.label.tolist()

In [ ]:
df.label.value_counts()

In [ ]:
chars = sorted(list(set(oobsents[0])))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
pred = LSTM_pred(model)

### Predict one character

In [ ]:
indx = random.randint(0,len(oobsents))
instr, y_true =  oobsents[indx], oobchars[indx]

res = pred.predict_completion_one(instr, SEQUENCE_LENGTH, 
                                  chars, char_indices, indices_char)
print("TRUE: {}| PREDICTED: {}".format(y_true,res))

### Multiple predictions: Run our model over bunch of iput data

In [ ]:
indx   = np.random.choice(np.arange(len(oobsents)), size=30000)
instr  = np.array(oobsents)[indx]
y_true = np.array(oobchars)[indx]

y_pred = []
for i in tqdm_notebook(range(len(instr))):
    res = pred.predict_completion_one(instr[i], SEQUENCE_LENGTH, 
                                      chars, char_indices, indices_char)
    y_pred.append(res)


In [ ]:
from multiprocessing import Pool, current_process, Queue

NUM_GPUS = 1
PROC_PER_GPU = 2    

queue = Queue()

def make_predict(pred,):
    gpu_id = queue.get()
    try:
        # run processing on GPU <gpu_id>
        ident = current_process().ident
        print('{}: starting process on GPU {}'.format(ident, gpu_id))
        res = pred.predict_completion_one(instr, SEQUENCE_LENGTH, 
                                          chars, char_indices, indices_char)
        print('{}: finished'.format(ident))
    finally:
        queue.put(gpu_id)
    return res

# initialize the queue with the GPU ids
for gpu_ids in range(NUM_GPUS):
    for _ in range(PROC_PER_GPU):
        queue.put(gpu_ids)

pool = Pool(processes=PROC_PER_GPU * NUM_GPUS)
batches = ['file{}.xyz'.format(x) for x in range(1000)]
res=[]
for out in pool.imap_unordered(make_predict, batches):
    res.append(out)
pool.close()
pool.join()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def confusion_matrix_heatmap(y_true, y_pred):
    '''
    Построение Confusion matrix (матрицы ошибок)

    Parameters
    ----------
    y_test: pandas.Series, numpy.array
        Целевая для обучающего набора
    y_pred: pandas.Series, numpy.array
        Значения целевой переменной, предсказанные классификатором
    '''
    rcParams['figure.figsize'] = 6, 4
    sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d', cmap='Greens')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
confusion_matrix_heatmap(y_true, y_pred)

In [ ]:
print(classification_report(y_true, y_pred))

### Call `predict_completions` instance method that wraps everything and allow us to predict multiple completions

In [ ]:
#repeatedly preparing input, asking our model for predictions and sampling from them!!!
pred.predict_completions(instr, 
                         SEQUENCE_LENGTH, 
                         chars, 
                         char_indices, 
                         indices_char)